In [1]:
Threads.nthreads()

12

In [4]:
include("BBVI_modeling.jl")

In [5]:
using RCall

R"""
load("data.RData")
"""
data = @rget data
Y = data[:Y]
Q = convert(Matrix{Int64}, data[:Q])
obs = DCMObs(Y, Q);

In [6]:
a0 = 1e-2
b0 = 1e-4
d0 = ones(size(obs.D[1], 1))
M = 1000
model = DCModel(obs, d0, a0, b0, M)

DCModel{Float64}(DCMObs{Int64}([0 1 … 0 0; 0 1 … 1 0; … ; 0 1 … 0 1; 1 0 … 1 1], [1 1; 1 1; … ; 0 0; 1 1], [[1 0 0 0; 1 1 0 0; 1 0 1 0; 1 1 1 1], [1 0 0 0; 1 1 0 0; 1 0 1 0; 1 1 1 1], [1 0; 1 1; 1 0; 1 1], [1 0; 1 1; 1 0; 1 1], [1; 1; 1; 1;;], [1 0; 1 1; 1 0; 1 1], [1 0 0 0; 1 1 0 0; 1 0 1 0; 1 1 1 1], [1 0 0 0; 1 1 0 0; 1 0 1 0; 1 1 1 1], [1; 1; 1; 1;;], [1 0; 1 0; 1 1; 1 1]  …  [1 0 0 0; 1 1 0 0; 1 0 1 0; 1 1 1 1], [1 0; 1 0; 1 1; 1 1], [1 0; 1 1; 1 0; 1 1], [1; 1; 1; 1;;], [1 0; 1 0; 1 1; 1 1], [1 0; 1 1; 1 0; 1 1], [1 0; 1 0; 1 1; 1 1], [1 0; 1 1; 1 0; 1 1], [1; 1; 1; 1;;], [1 0 0 0; 1 1 0 0; 1 0 1 0; 1 1 1 1]]), [1.0, 1.0, 1.0, 1.0], 0.01, 0.0001, [[0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25]  …  [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25

In [12]:
function sample_seq(model)
    @inbounds for i in 1:1000
        sample_variational_distribution(model, sample_Z=true, idx_Z=i)
    end
end

function sample_par(model)
    Threads.@threads for i in 1:1000
        sample_variational_distribution(model, sample_Z=true, idx_Z=i)
    end
end

sample_par (generic function with 2 methods)

In [13]:
using BenchmarkTools

@benchmark sample_seq($model)

BenchmarkTools.Trial: 282 samples with 1 evaluation.
 Range (min … max):  16.753 ms … 30.346 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.187 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.784 ms ±  1.879 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅██▇▆▄▁▁                                                     
  █████████▇▇▄▇▇▆▁▄▁▄▁▁▁▄▄▁▁▁▁▁▆▁▁▁▁▄▁▁▄▁▁▁▁▁▄▁▄▄▁▁▄▄▁▄▄▁▆▄▄▆ ▆
  16.8 ms      Histogram: log(frequency) by time      25.1 ms <

 Memory estimate: 125.00 KiB, allocs estimate: 1000.

In [14]:
@benchmark sample_par($model)

BenchmarkTools.Trial: 580 samples with 1 evaluation.
 Range (min … max):  6.803 ms … 12.863 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.582 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.611 ms ±  1.183 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▅▄▁▃ ▃ ▄▄▃    ▁  ▂▂▄▁ ▃█▄ ▁▄▂ ▁                            
  █████▆█████▇▇▇███▆███████████████▅█▇▆▄▄▆▅▄▄▅▅▁▃▃▄▄▁▃▃▁▃▃▃▄ ▅
  6.8 ms         Histogram: frequency by time        11.9 ms <

 Memory estimate: 135.27 KiB, allocs estimate: 1061.